In [1]:
import nfl_data_py as nfl
import pandas as pd
import seaborn as sns

In [2]:
years = [2022]

pbp = nfl.import_pbp_data(years)
roster = nfl.import_rosters(years)

2022 done.
Downcasting floats.


In [47]:
points_per_pass_tds = 4
points_per_pass_yds = .04
points_per_pass_int = -2
points_per_touchdowns = 6
points_per_receptions = .5
points_per_yards = .1

In [76]:
pbp_fantasy = pbp[['play_type', 'game_id', 'season_type', 'week', 'posteam', 'desc', 'yards_gained', 'complete_pass', 'touchdown', 'first_down','interception', 'passer_player_id', 'passer_player_name', 'fantasy_player_id', 'fantasy_player_name']]
pbp_fantasy = pbp_fantasy[(pbp_fantasy['play_type'] == 'run') |(pbp_fantasy['play_type'] == 'pass')]
pbp_fantasy = pbp_fantasy[pbp_fantasy['season_type'] == 'REG']

In [77]:
filtered_roster = roster[['player_id', 'position']]
pos_dict = dict(filtered_roster.values)

pos_dict

{'00-0029747': 'DB',
 '00-0032560': 'DB',
 '00-0032933': 'DB',
 '00-0033890': 'DB',
 '00-0033568': 'DB',
 '00-0034751': 'DB',
 '00-0035236': 'DB',
 '00-0035285': 'DB',
 '00-0035705': 'DB',
 '00-0036043': 'DB',
 '00-0036077': 'DB',
 '00-0036189': 'DB',
 '00-0036884': 'DB',
 '00-0036667': 'DB',
 '00-0037747': 'DB',
 '00-0027949': 'DL',
 '00-0032871': 'DL',
 '00-0035247': 'DL',
 '00-0035248': 'DL',
 '00-0035541': 'DL',
 '00-0035343': 'DL',
 '00-0036371': 'DL',
 '00-0036373': 'DL',
 '00-0035928': 'DL',
 '00-0037551': 'DL',
 '00-0032127': 'LB',
 '00-0032892': 'LB',
 '00-0032437': 'LB',
 '00-0033580': 'LB',
 '00-0034296': 'LB',
 '00-0034473': 'LB',
 '00-0034490': 'LB',
 '00-0036356': 'LB',
 '00-0035984': 'LB',
 '00-0036933': 'LB',
 '00-0036896': 'LB',
 '00-0037815': 'LB',
 '00-0037751': 'LB',
 '00-0037330': 'LB',
 '00-0027993': 'OL',
 '00-0029316': 'OL',
 '00-0030585': 'OL',
 '00-0032222': 'OL',
 '00-0032132': 'OL',
 '00-0033560': 'OL',
 '00-0033511': 'OL',
 '00-0033631': 'OL',
 '00-0034762'

In [78]:
qb_df = pbp_fantasy
qb_df['position'] = qb_df['passer_player_id'].map(pos_dict)
qb_pass_df = qb_df[qb_df['position']=='QB']
qb_pass_df = qb_pass_df[qb_pass_df['play_type'] == 'pass']
qb_pass_df['fantasy_pts'] =  (qb_pass_df['yards_gained'] * points_per_pass_yds) + (qb_pass_df['touchdown'] * points_per_pass_tds) + (qb_pass_df['interception'] * points_per_pass_int)
qb_pass_df['fantasy_player_id'] = qb_pass_df['passer_player_id'] 
qb_pass_df['fantasy_player_name'] = qb_pass_df['passer_player_name'] 
qb_pass_df = qb_pass_df.drop(columns=['passer_player_id', 'passer_player_name'])

qb_pass_df.head(5)

,play_type,game_id,season_type,week,posteam,desc,yards_gained,complete_pass,touchdown,first_down,interception,fantasy_player_id,fantasy_player_name,position,fantasy_pts
3,pass,2022_01_BAL_NYJ,REG,1,NYJ,"(14:29) (No Huddle, Shotgun) 19-J.Flacco pass ...",0.0,0.0,0.0,0.0,0.0,00-0026158,J.Flacco,QB,0.00
5,pass,2022_01_BAL_NYJ,REG,1,NYJ,"(14:01) (No Huddle, Shotgun) 19-J.Flacco pass ...",0.0,0.0,0.0,0.0,0.0,00-0026158,J.Flacco,QB,0.00
7,pass,2022_01_BAL_NYJ,REG,1,BAL,(13:42) 8-L.Jackson pass short right to 7-R.Ba...,4.0,1.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.16
8,pass,2022_01_BAL_NYJ,REG,1,BAL,"(13:21) (No Huddle, Shotgun) 8-L.Jackson pass ...",4.0,1.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.16
11,pass,2022_01_BAL_NYJ,REG,1,BAL,(11:19) (Shotgun) 8-L.Jackson pass short left ...,8.0,1.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.32


In [79]:
qb_run_df = pbp_fantasy
qb_run_df['position'] = qb_run_df['fantasy_player_id'].map(pos_dict)
qb_run_df = qb_run_df[qb_run_df['play_type'] == 'run']
qb_run_df = qb_run_df[qb_run_df['position'] == 'QB']
qb_run_df['passer_player_id'] = qb_run_df['fantasy_player_id']
qb_run_df = qb_run_df.drop(columns=['passer_player_id', 'passer_player_name'])
qb_run_df['fantasy_pts'] =  (qb_run_df['touchdown'] * points_per_touchdowns) + (qb_run_df['complete_pass'] * points_per_receptions) + (qb_run_df['yards_gained'] * points_per_yards)

points_per_touchdowns = 6
points_per_receptions = .5
points_per_yards = .1

qb_run_df.head(5)

,play_type,game_id,season_type,week,posteam,desc,yards_gained,complete_pass,touchdown,first_down,interception,fantasy_player_id,fantasy_player_name,position,fantasy_pts
9,run,2022_01_BAL_NYJ,REG,1,BAL,(12:41) (Shotgun) 8-L.Jackson right tackle to ...,4.0,0.0,0.0,1.0,0.0,00-0034796,L.Jackson,QB,0.4
89,run,2022_01_BAL_NYJ,REG,1,BAL,(14:15) (Shotgun) 8-L.Jackson scrambles left e...,8.0,0.0,0.0,1.0,0.0,00-0034796,L.Jackson,QB,0.8
99,run,2022_01_BAL_NYJ,REG,1,BAL,(9:49) (Shotgun) 8-L.Jackson left end to NYJ 2...,6.0,0.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.6
102,run,2022_01_BAL_NYJ,REG,1,BAL,(8:03) (Shotgun) 8-L.Jackson scrambles left en...,1.0,0.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.1
144,run,2022_01_BAL_NYJ,REG,1,NYJ,(8:10) (No Huddle) 19-J.Flacco up the middle t...,0.0,0.0,0.0,0.0,0.0,00-0026158,J.Flacco,QB,0.0


In [80]:
combined_qb_df = pd.concat([qb_run_df, qb_pass_df])

combined_qb_df

,play_type,game_id,season_type,week,posteam,desc,yards_gained,complete_pass,touchdown,first_down,interception,fantasy_player_id,fantasy_player_name,position,fantasy_pts
9,run,2022_01_BAL_NYJ,REG,1,BAL,(12:41) (Shotgun) 8-L.Jackson right tackle to ...,4.0,0.0,0.0,1.0,0.0,00-0034796,L.Jackson,QB,0.40
89,run,2022_01_BAL_NYJ,REG,1,BAL,(14:15) (Shotgun) 8-L.Jackson scrambles left e...,8.0,0.0,0.0,1.0,0.0,00-0034796,L.Jackson,QB,0.80
99,run,2022_01_BAL_NYJ,REG,1,BAL,(9:49) (Shotgun) 8-L.Jackson left end to NYJ 2...,6.0,0.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.60
102,run,2022_01_BAL_NYJ,REG,1,BAL,(8:03) (Shotgun) 8-L.Jackson scrambles left en...,1.0,0.0,0.0,0.0,0.0,00-0034796,L.Jackson,QB,0.10
144,run,2022_01_BAL_NYJ,REG,1,NYJ,(8:10) (No Huddle) 19-J.Flacco up the middle t...,0.0,0.0,0.0,0.0,0.0,00-0026158,J.Flacco,QB,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47823,pass,2022_18_TEN_JAX,REG,18,TEN,(3:01) (Shotgun) 11-J.Dobbs sacked at TEN 27 f...,0.0,0.0,1.0,0.0,0.0,00-0033949,J.Dobbs,QB,4.00
47828,pass,2022_18_TEN_JAX,REG,18,TEN,(2:06) (Shotgun) 11-J.Dobbs pass short middle ...,5.0,1.0,0.0,1.0,0.0,00-0033949,J.Dobbs,QB,0.20
47831,pass,2022_18_TEN_JAX,REG,18,TEN,(1:55) (Shotgun) 11-J.Dobbs to TEN 45 for -8 y...,-5.0,0.0,0.0,0.0,0.0,00-0033949,J.Dobbs,QB,-0.20
47834,pass,2022_18_TEN_JAX,REG,18,TEN,(1:46) (Shotgun) 11-J.Dobbs pass short middle ...,4.0,1.0,0.0,0.0,0.0,00-0033949,J.Dobbs,QB,0.16


In [81]:
qb_weekly_df = combined_qb_df.groupby(by=['fantasy_player_id', 'fantasy_player_name', 'posteam', 'week', 'position']).sum()

qb_weekly_df.head(50)

yards_gained  \
fantasy_player_id fantasy_player_name posteam week position                 
00-0019596        T.Brady             TB      1    QB               195.0   
                                              2    QB               188.0   
                                              3    QB               251.0   
                                              4    QB               373.0   
                                              5    QB               353.0   
                                              6    QB               229.0   
                                              7    QB               277.0   
                                              8    QB               306.0   
                                              9    QB               272.0   
                                              10   QB               258.0   
                                              12   QB               231.0   
                                              13   QB               274.0   
                                              14   QB               254.0   
                                              15   QB               305.0   
                                              16   QB               283.0   
                                              17   QB               417.0   
                                              18   QB                84.0   
00-0023459        A.Rodgers           GB      1    QB               162.0   
                                              2    QB               222.0   
                                              3    QB               248.0   
                                              4    QB               245.0   
                                              5    QB               207.0   
                                              6    QB               210.0   
                                              7    QB               194.0   
                                              8    QB               201.0   
                                              9    QB               324.0   
                                              10   QB               215.0   
                                              11   QB               217.0   
                                              12   QB               123.0   
                                              13   QB               190.0   
                                              15   QB               207.0   
                                              16   QB               243.0   
                                              17   QB               145.0   
                                              18   QB               198.0   
00-0026143        M.Ryan              IND     1    QB               352.0   
                                              2    QB               164.0   
                                              3    QB               182.0   
                                              4    QB               323.0   
                                              5    QB               201.0   
                                              6    QB               385.0   
                                              7    QB               229.0   
                                              10   QB               248.0   
                                              11   QB               186.0   
                                              12   QB               202.0   
                                              13   QB               204.0   
                                              15   QB               171.0   
00-0026158        J.Flacco            NYJ     1    QB               297.0   
                                              2    QB               299.0   
                                              3    QB               252.0   
                                              14   QB                -5.0   

                                                        